In [1]:
import numpy as np 
import pandas as pd
import random
import glob
import ipywidgets as widgets
pd.options.display.max_rows = 999
from PIL import Image
from ipywidgets import Button, Layout
from IPython.display import Markdown, display


In [2]:
df = pd.read_excel("Outputs.xlsx").set_index("FileName")
def printmd(string):
    display(Markdown(string))

In [3]:
jpegfiles = []
for file in glob.glob("Favorites/*.jpeg"):
    jpegfiles.append(file)
    
jpegfiles.sort()

uniques = np.concatenate((df['PersonA'].unique(), df['PersonB'].unique(), df['PersonC'].unique(), df['PersonD'].unique()))
df4 = pd.DataFrame(uniques)
Uni = df4[0].unique()

Uni = Uni.tolist()
while (np.nan in Uni):
    Uni.remove(np.nan)

In [7]:
Person1 = widgets.Combobox(
    placeholder='Choose Someone',
    options=Uni,
    description='Person 1:',
    disabled=False
)

Person2 = widgets.Combobox(
    placeholder='Choose Someone',
    options=Uni,
    description='Person 2:',
    disabled=False
)

Person3 = widgets.Combobox(
    placeholder='Choose Someone',
    options=Uni,
    description='Person 3:',
)

Person4 = widgets.Combobox(
    placeholder='Choose Someone',
    options=Uni,
    description='Person 4:',
)


ImageDisplay = widgets.Image(
    format='png',
    width="300px",
    height="400px",
)

choice = ""


toggle = widgets.ToggleButton(description='Find Photo!')
toggle2 = widgets.ToggleButton(description='View Photo in Full')


Img = widgets.Output()
Txt = widgets.Output()
Txt2 = widgets.Output()



in_box = widgets.VBox([Person1,Person2,Person3,Person4,toggle,toggle2,Txt])


box_layout = Layout(display='flex',
                    align_items='center',
                    width='50%')
                    
in_box.layout = box_layout

out_box = widgets.VBox([Txt2, Img])

fin_box = widgets.HBox([in_box,out_box])
fin_box.layout.height = '500px'
fin_box.justify_content='flex-end'
fin_box.layout.align_items = 'stretch'



def fun(obj):
    txt_out = ""
    global choice
    global file
    fits = []
    req = [Person1.value, Person2.value, Person3.value, Person4.value]
    while ("" in req):
        req.remove("")
            

    with Img:
        Img.clear_output()

        if (len(req)>0):
            for file in jpegfiles:
                if all(elem in df.loc[file].values.tolist() for elem in req):
                    fits.append(file)
            if (len(fits) > 0):
                choice = random.choice(fits)
                ImageDisplay.value = open(choice, "rb").read()
                display(ImageDisplay)

        else:
            fits = jpegfiles
            if (len(fits) > 0):
                choice = random.choice(fits)
                ImageDisplay.value = open(choice, "rb").read()
                display(ImageDisplay)
            
    with Txt:
        
        Txt.clear_output()
        printmd("Requirements:")
        if (len(req)==0):
            txt_out = "None"
            printmd(txt_out)
        if (len(fits)>0):
            for i in req:
                printmd(i)
                
        else:
            txt_out = "No Image Found"
            printmd(txt_out)
            
    with Txt2:
        
        Txt2.clear_output()
        st_out = ""
        if (txt_out != "No Image Found"):
            printmd("Pictured: (1) of (" + str(len(fits)) + ").")
            for i in df.loc[choice].values.tolist():
                if type(i) is str:
                    if st_out != "":
                        st_out += ", " + i
                    else:
                        st_out += i 
        else:
            st_out = "No Image"
        printmd(st_out)

        
def shower(obj):
    im = Image.open(choice) 
    im.show()
      


toggle.observe(fun, 'value')

toggle2.observe(shower, 'value')

fin_box



In [5]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')